In [1]:
import pickle, tqdm, time, os, tensorflow_hub as hub, numpy as np, pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
# load & cache tensorflow model
embed = hub.load("C:/Users/AlamjeetSingh/Downloads/temp/universal-sentence-encoder-large_5.tar/universal-sentence-encoder-large_5")
embed(['testing'])
print('model cached')

model cached


In [ ]:
# load data df & embeddings
df = pd.read_pickle('data/clean_data.pkl')
title_embeddings = np.load('data/title_embeddings.npz')['a']
# text_embeddings = np.load('data/texts_embeddings.npz')['a']


**STEPS:**
1. Query --> query_vector = Q
2. Query --> find similar docs, return top N most similar docs
3. User Faves Doc A --> Replace Q with Q' = Q + (K)(A - Q) where K is param between 0 & 1 we choose
3. User Hides Doc B --> Replace Q' with Q'' = (1 + K)(Q') - (K)(B)

In [ ]:
SENSITIVITY = 0.75
EMBED_TYPE = 'titles'

In [ ]:
def getMostSimilar(query_vector, embeds=title_embeddings, docsdf=df.copy(),filter_bad=True):
    scores = query_vector.dot(embeds.T).flatten()
    docsdf['similarity_score'] = scores
    docsdf = docsdf.sort_values('similarity_score', ascending=False).copy()
    return docsdf

In [ ]:
def update_embedding(q_vector, feedback_vector, feedback):
    # New q vector = (1 - feedback*SENSITIVITY)*(q_vector)  + feedback*SENSITIVITY*(feedback_vector)
    new_q = (1 - feedback*SENSITIVITY)*q_vector + feedback*SENSITIVITY*feedback_vector
    return new_q

In [ ]:
def giveFeedback(post_id, feedback):
    feedback_doc = df[df['ids'] == post_id][EMBED_TYPE].values[0]
    feedback_vector = embed([feedback_doc]).numpy()
    new_q = update_embedding(Q, feedback_vector, feedback)
    return new_q

In [ ]:
QUERY = 'password'
Q = embed([QUERY]).numpy()

In [ ]:
sim = getMostSimilar(Q)
sim[['ids','titles','similarity_score']].head(50)

In [ ]:
Q = giveFeedback(post_id='gxydco', feedback=1)

In [ ]:
sim = getMostSimilar(Q)
sim[['ids','titles','similarity_score']].head(50)

In [ ]:
sim['similarity_score'].plot(kind='hist')

In [ ]:
sim['similarity_score'].describe()